In [1]:
import geemap
import ee

from ipyleaflet import basemaps
import datetime

# Air Pollution over Indonesia

Sulfur Dioxide Data is obtained from Finnish Meteorological Institute (https://silam.fmi.fi/) overlaid with Coal Power Plants data from World Resource Institute (WRI). Units in ug/m^3 and values below 10 ug/m^3 are not shown on the map. This map is updated every hour. For more information, please visit https://josefmtd.com/.

In [2]:
# Get the observed datetime
now = datetime.datetime.utcnow()
obs = datetime.datetime(now.year, now.month, now.day, now.hour)

print('Datetime :', obs.strftime("%Y-%m-%d %H:%M UTC"))

Datetime : 2021-07-30 10:00 UTC


In [3]:
# Create a map service
Map = geemap.Map(center=(0,120), zoom=5,
    min_zoom=5, max_zoom=12,
    basemap=basemaps.CartoDB.DarkMatter,
    add_google_map=False
)

In [4]:
# Obtain NO2 data from SILAM TDS Catalog exported to Google Cloud Storage
fname = f'SO2_{obs.strftime("%Y%m%dT%H")}.tif'
var = ee.Image.loadGeoTIFF(f'gs://silam-neonet-rasters/{fname}')

# Resample and only show values above 10 ug/m^3
data = var.resample().reproject(ee.Projection('EPSG:4326'), scale=1000)
data = data.updateMask(data.gt(10))

# Add Coal Power Plant from WRI's Global Power Plant Database
indo_power = ee.FeatureCollection('WRI/GPPD/power_plants') \
    .filter(ee.Filter.eq('country', 'IDN')) \
    .filter(ee.Filter.eq('fuel1', 'Coal'))

In [5]:
# Limit the area around Indonesia
bbox = ee.Geometry.BBox(80.0, -15.0, 160.0, 15.0)

# Visualization Parameters
vmin = 0.0
vmax = 250.0
palette = ['blue', 'yellow', 'red', 'purple']

vis_params = {
    'min' : vmin,
    'max' : vmax,
    'palette' : palette,
    'opacity' : 0.5,
}

# Add Coal Power Plant data and Sulfur Dioxide data
Map.addLayer(data.clip(bbox), vis_params, 'SO2')
Map.addLayer(indo_power, {'color' : 'ff0000'}, 'Coal Power Plant')
Map.add_colorbar_branca(colors=palette, vmin=vmin, vmax=vmax,
    layer_name='SO2')
Map.addLayerControl()
Map

Map(center=[0, 120], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…